# Check Consensus Nodes

In [3]:
import json
import boto3
import requests
from requests.auth import HTTPBasicAuth

networks = {
    "goerli": {
        "requires_auth": True
    }
}

for network, requires_auth in networks.items():
    
    url = f"https://nodes.casimir.co/eth/{network}/eth/v1/node/syncing"

    headers = {'content-type': 'application/json'}

    auth = None
    if requires_auth:
        client = boto3.client('secretsmanager')
        response = client.get_secret_value(SecretId='casimir-rpc-credentials')
        secret = json.loads(response['SecretString'])
        auth = HTTPBasicAuth(secret['username'], secret['password'])

    response = requests.get(url, headers=headers, auth=auth)

    if response.status_code != 200:
        print(f"Casimir {network} consensus node is not responding")
    else:        
        data = response.json()['data']
        if bool(data['is_syncing']) == False:
            print(f"Casimir {network} consensus node is fully synced")
        else:
            print(f"Casimir {network} consensus node is syncing. Current slot: {int(data['head_slot'])}. Highest slot: {int(data['head_slot']) + int(data['sync_distance'])}")
        



Casimir goerli consensus node is syncing. Current slot: 5943238. Highest slot: 6662784
